## Dataloader

## Create index DataFrame

In [1]:
from dataset import load_pdbbind_data_index
import sys

if sys.platform == 'linux':
    index_location = '/home/longyuxi/Documents/mount/pdbbind-dataset/index/INDEX_refined_data.2020'
else:
    raise Exception

index = load_pdbbind_data_index(index_location)

## For debug purposes

# index = index[:400]
index

,PDB code,resolution,release year,-logKd/Ki,Kd/Ki,reference,ligand name
0,2r58,2.00,2007,2.00,Kd=10mM,2r58.pdf,MLY
1,3c2f,2.35,2008,2.00,Kd=10.1mM,3c2f.pdf,PRP
2,3g2y,1.31,2009,2.00,Ki=10mM,3g2y.pdf,GF4
3,3pce,2.06,1998,2.00,Ki=10mM,3pce.pdf,3HP
4,4qsu,1.90,2014,2.00,Kd=10mM,4qsu.pdf,TDR
...,...,...,...,...,...,...,...
5311,4f3c,1.93,2013,11.82,Ki=1.5pM,4f1w.pdf,BIG
5312,5bry,1.34,2015,11.82,Ki=0.0015nM,5bry.pdf,4UY
5313,1sl3,1.81,2004,11.85,Ki=1.4pM,1sl3.pdf,170
5314,1ctu,2.30,1995,11.92,Ki=1.2pM,1ctu.pdf,ZEB


In [2]:
index.loc[2, '-logKd/Ki']

2.0

## Dataloader

In [3]:
import numpy as np
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from models import SusNet

from dataset import ProteinPersistenceHomologyDataset, CollapsedPairwiseOppositionDataset, ProteinHomologyDataModule

In [4]:
cpod = CollapsedPairwiseOppositionDataset(index)

ph, ba = cpod[1]

Global seed set to 123


In [5]:
ph.shape

torch.Size([36, 201])

In [6]:
# from pytorch_lightning.loggers import WandbLogger

# wandb_logger = WandbLogger(project="Topos")

In [7]:
sn = SusNet()
# train_loader = DataLoader(CollapsedPairwiseOppositionDataset(index), batch_size=5, num_workers=12)
datamodule = ProteinHomologyDataModule(index, batch_size=32)
# trainer = pl.Trainer(max_epochs=10, logger=wandb_logger, log_every_n_steps=1)
if sys.platform == 'linux':
    trainer = pl.Trainer(max_epochs=10, accelerator='gpu', devices=1)
else:
    raise Exception

trainer.fit(sn, datamodule=datamodule)

Global seed set to 123
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type       | Params
------------------------------------------
0  | c1        | Sequential | 15.7 K
1  | c2        | Sequential | 62.4 K
2  | pool      | AvgPool1d  | 0     
3  | dropout   | Dropout1d  | 0     
4  | c3        | Sequential | 124 K 
5  | c4        | Sequential | 249 K 
6  | d1        | Sequential | 29.5 M
7  | d2        | Sequential | 4.2 M 
8  | d3        | Sequential | 4.2 M 
9  | dropout2  | Dropout1d  | 0     
10 | fc        | Linear     | 2.0 K 
11 | criterion | MSELoss    | 0     
------------------------------------------
38.3 M    Trainable params
0         Non-trainable params
38.3 M    Total params
153.359   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## TODO: empirically determine whether it is the same to translate vs not to translate the ligand wrt protein